# DataBento Downloader

In [ ]:
''' DataBento Downloader
    Downloads the tickers within the tickers[] list. Saves them locally to a CSV file.
'''
### Region Imports ###
import databento as db
from dotenv import load_dotenv
import os
import pandas as pd
from datetime import datetime
### End Region Imports ###

# Initialize Data Bento client
client = db.Historical(os.getenv('databento_api_key'))

# Example tickers and date range
tickers = ['SPY']
start_date = '2020-01-01'
end_date = '2024-09-01'

for ticker in tickers:
    dataset = client.timeseries.get_range(
        dataset="XNAS.ITCH",
        symbols=ticker,
        start=start_date,
        end=end_date,
        schema='ohlcv-1d'
    )
    dataset.to_csv(f'databento/downloads/{ticker}_data.csv')


# Converter Method
Working as of 26SEP24

In [2]:
import pandas as pd
from datetime import datetime
import zipfile
import os

ticker = 'SPY'
def convert_to_lean_format(csv_file, ticker, frequency='daily'):
    """
    Converts a CSV file containing stock data into a format compatible with LEAN Local CLI Framework.
    Args:
        csv_file (str): The path to the input CSV file containing stock data.
        ticker (str): The stock ticker symbol.
        frequency (str, optional): The frequency of the data. Can be 'daily', 'hourly', or 'minute'. Defaults to 'daily'.
    Returns:
        None: The function saves the converted data to a file in the appropriate directory based on the frequency. Located within project directory.
    """
    
    df = pd.read_csv(csv_file)
    df['date'] = pd.to_datetime(df.pop('ts_event')).dt.strftime('%Y%m%d %H:%M')
    df = df[['date', 'open', 'high', 'low', 'close', 'volume']]
    
    # Conversion from dollars to deci-cents
    df[['open', 'high', 'low', 'close']] = (df[['open', 'high', 'low', 'close']] * 10000).astype(int)
    
    # Data type sorting for directory saving for csv
    if frequency == 'daily':
        output_dir = 'data/equity/usa/daily/'
        output_file = f"{output_dir}{ticker.lower()}.csv"
    elif frequency == 'hourly':
        output_dir = 'data/equity/usa/hourly/'
        output_file = f"{output_dir}{ticker.lower()}.csv"
    else:
        output_dir = 'data/equity/usa/minute/'
        output_file = f"{output_dir}{ticker.lower()}.zip"
    
    df.to_csv(output_file, index=False, header=False)
    
    # Zip File
    zip_file = os.path.join(output_dir, f'{ticker}.zip')
    # Create a zip file and add the csv file to it
    with zipfile.ZipFile(zip_file, 'w') as zf:
        zf.write(csv_file)

    # Optionally, remove the csv file after zipping (if you want the zip to be the only output)
    #os.remove(csv_file)

    print(f"{csv_file} has been successfully zipped into {zip_file}.")

In [3]:
csv_file = f'databento/downloads/{ticker}_data.csv'
frequency = 'daily'
convert_to_lean_format(csv_file, ticker, frequency)

databento/downloads/SPY_data.csv has been successfully zipped into data/equity/usa/daily/SPY.zip.


#### Combined file, downloads then converts using databento

In [1]:
import databento as db
import pandas as pd
from datetime import datetime

def convert_to_lean_format(csv_file, ticker, frequency='daily'):
    """
    Converts a CSV file containing stock data into a format compatible with LEAN Local CLI Framework.
    Args:
        csv_file (str): The path to the input CSV file containing stock data.
        ticker (str): The stock ticker symbol.
        frequency (str, optional): The frequency of the data. Can be 'daily', 'hourly', or 'minute'. Defaults to 'daily'.
    Returns:
        None: The function saves the converted data to a file in the appropriate directory based on the frequency. Located within project directory.
    """
    
    df = pd.read_csv(csv_file)
    df['date'] = pd.to_datetime(df.pop('ts_event')).dt.strftime('%Y%m%d %H:%M')
    df = df[['date', 'open', 'high', 'low', 'close', 'volume']]
    
    if frequency == 'daily':
        output_file = f"data/equity/usa/daily/{ticker.lower()}.csv"
    elif frequency == 'hourly':
        output_file = f"data/equity/usa/hourly/{ticker.lower()}.csv"
    else:
        output_file = f"data/equity/usa/minute/{ticker.lower()}.zip"
    
    df.to_csv(output_file, index=False, header=False)

def download_and_convert(tickers, start_date, end_date, frequency='daily'):
    for ticker in tickers:
        dataset = client.timeseries.get_range(
            dataset="XNAS.ITCH",
            symbols=ticker,
            start=start_date,
            end=end_date,
            schema='ohlcv-1d'
        )
        csv_file = f'databento/downloads/{ticker}_data.csv'
        dataset.to_csv(csv_file)

        # Convert to QuantConnect format
        convert_to_lean_format(csv_file, ticker, frequency)

# Example ticker list and date range
ticker_list = ['QQQ']
download_and_convert(ticker_list, '2023-01-01', '2023-09-01', 'daily')


NameError: name 'client' is not defined